In [64]:
out = "/home/mdonovan/pancreas_scRNA_map/Table/Final/"

# Table 1

Description of samples in study, %PDX1+/NKX6-1, and UUIDs for all sequencing data

In [65]:
table_count = 1

In [66]:
t1 = read.csv("../Table/Table S1_Clinical and Molecular Metadata_V03.csv")

In [67]:
t1 = t1[! t1$UDID == "PPC_069", ]

In [68]:
t1$Bulk_ATAC.seq_UUID <- NULL
t1$bulk_RNAseq_UUID   <- NULL

In [69]:
colnames(t1) <- c("Subject_UUID", "Sex", "Ethinicity", "iPSCORE_Family", "Cell_type", "UDID", "PDX1_pos", "PDX1.NKX61_pos", "WGS_UUID",
                  "scRNAseq_live_UUID", "scRNAseq_Cryporeserved_UUID", "scRNAseq_Cryporeserved_Pool_scheme", "snATACseq._UUID",
                  "snATACseq._Pool_scheme")

In [70]:
write.csv(t1, file = paste(out, "Table_", table_count, ".csv", sep = ""))

# Table 2
Differentially expressed genes from each cell type

In [72]:
table_count = 2

In [73]:
in.f = "/projects/PPC/analysis/ppc_pilot/data/aggr_live_then_integrate/"

iPSC_markers        = read.table(paste(in.f, "iPSC_markers.txt", sep = ""))
repl_markers        = read.table(paste(in.f, "repl_markers.txt", sep = ""))
ppc_markers         = read.table(paste(in.f, "all.ppc_markers.txt", sep = ""))
Endocrine_markers   = read.table(paste(in.f, "Endocrine_markers.txt", sep = ""))
mesenchymal_markers = read.table(paste(in.f, "mesenchymal_markers.txt", sep = ""))
endothelial_markers = read.table(paste(in.f, "endothelial_markers.txt", sep = ""))
endothelial_markers = read.table(paste(in.f, "non.ppc.c4_markers.txt", sep = ""))

In [74]:
AnnotateTableMarker <- function(df, celltype){
    
    df$marker_gene_name = rownames(df)
    rownames(df) = NULL
    df$cell_type <- celltype
    df = df[, c("cell_type", "marker_gene_name", "p_val", "avg_logFC", "p_val_adj")]
    
    return(df)
    
}

In [75]:
t2 = rbind(AnnotateTableMarker(iPSC_markers, "iPSC"),
           AnnotateTableMarker(repl_markers, "Replicating cells"),
           AnnotateTableMarker(ppc_markers, "PPCs (no PPC-sc4)"),
           AnnotateTableMarker(Endocrine_markers, "Endocrine cells"),
           AnnotateTableMarker(mesenchymal_markers, "Mesenchymal cells"),
           AnnotateTableMarker(endothelial_markers, "Endothelial cells"),
           AnnotateTableMarker(ppc_markers, "PPC-sc4")
     )

In [76]:
write.csv(t2, file = paste(out, "Table_", table_count, ".csv", sep = ""))

# NO LONGER INCLUDED: Table 3
GO enrichment of marker genes from each cell type

In [125]:
table_count = 3

In [126]:
in.f = "/projects/PPC/analysis/ppc_pilot/data/aggr_live_then_integrate/"

gowall_ipsc        = read.table(paste(in.f, "gowall_ipsc.txt", sep = ""))
gowall_repl        = read.table(paste(in.f, "gowall_repl.txt", sep = ""))
gowall_mesenchyme  = read.table(paste(in.f, "gowall_mesenchyme.txt", sep = ""))
gowall_endocrine   = read.table(paste(in.f, "gowall_endocrine.txt", sep = ""))
gowall_endothelial = read.table(paste(in.f, "gowall_endothelial.txt", sep = ""))
gowall_ppc         = read.table(paste(in.f, "gowall_ppc.txt", sep = ""))

In [127]:
AnnotateTableGO <- function(df, celltype){
    
    rownames(df) = NULL
    df$cell_type <- celltype
    df = df[, c("cell_type", "category", "over_represented_pvalue", "under_represented_pvalue", "numDEInCat", "numInCat", "term", "ontology", "pvalue", "bonferroni")]
    return(df)
    
}

In [128]:
t3 = rbind(AnnotateTableGO(gowall_ipsc, "iPSC"),
           AnnotateTableGO(gowall_repl, "Replicating cells"),
           AnnotateTableGO(gowall_ppc, "PPCs (1, 2, 3, and 4)"),
           AnnotateTableGO(gowall_endocrine, "Endocrine cells"),
           AnnotateTableGO(gowall_mesenchyme, "Mesenchyme cells"),
           AnnotateTableGO(gowall_endothelial, "Endothelial cells")
     )

In [129]:
write.csv(t3, file = paste(out, "Table_", table_count, ".csv", sep = ""))

# Table 3

PPC genes that linearly change over time

In [77]:
in.f = "/projects/PPC/analysis/ppc_pilot/data/aggr_live_then_integrate/"
lm_fit_coefs = read.table(paste(in.f, "ppc_fit_coefs_frazer_ppc_allexpressedgenes_191212.txt", sep = ""))

In [78]:
table_count = 3

In [79]:
# in.f = "/projects/PPC/analysis/ppc_pilot/data/aggr_live_then_integrate/"
# lm_fit_coefs = read.table(paste(in.f, "ppc_fit_coefs_frazer_ppc_allexpressedgenes.txt", sep = ""))

In [80]:
lm_fit_coefs$Type = NA
lm_fit_coefs[lm_fit_coefs$q_value < 0.05 & lm_fit_coefs$estimate > 0, ]$Type = "Class_1"
lm_fit_coefs[lm_fit_coefs$q_value < 0.05 & lm_fit_coefs$estimate < 0, ]$Type = "Class_2"

In [81]:
lm_fit_coefs = lm_fit_coefs[order(lm_fit_coefs$q_value), ]

In [82]:
rownames(lm_fit_coefs) <- NULL

In [83]:
table(lm_fit_coefs$Type)


Class_1 Class_2 
    222      80 

In [84]:
write.csv(lm_fit_coefs, file = paste(out, "Table_", table_count, ".csv", sep = ""))

# Table 4
PPC genes that change non-linearly over time (MFuzz membership matrix)

In [85]:
table_count = 4

In [86]:
in.f = "/projects/PPC/analysis/ppc_pilot/data/aggr_live_then_integrate/"
mfuzz = read.table(paste(in.f, "mfuzz_k10_b4_memberships_191213.txt", sep = ""))

In [87]:
mfuzz$Type = NA
mfuzz[mfuzz$maxCol == 1 & mfuzz$real == TRUE, ]$Type = "Class_3"
mfuzz[mfuzz$maxCol == 6 & mfuzz$real == TRUE, ]$Type = "Class_4"
mfuzz = mfuzz[order(mfuzz$Type), ]
mfuzz$maxCol <- NULL
mfuzz$real <- NULL

In [88]:
colnames(mfuzz) = c("Cluster_1", "Cluster_2", "Cluster_3", "Cluster_4", "Cluster_5", 
                    "Cluster_6", "Cluster_7", "Cluster_8", "Cluster_9", "Cluster_10", "Type")

In [89]:
table(mfuzz$Type)


Class_3 Class_4 
    611     613 

In [90]:
write.csv(mfuzz, file = paste(out, "Table_", table_count, ".csv", sep = ""))

# Table 5

In [91]:
table_count = 5

In [92]:
in.f = "/projects/PPC/analysis/ppc_pilot/data/aggr_live_then_integrate/"
go_up   = read.table(paste(in.f, "gowall_ppc_up_191216.txt", sep = ""))
go_down = read.table(paste(in.f, "gowall_ppc_down_191216.txt", sep = ""))
go_mid.up = read.table(paste(in.f, "gowall_ppc_mid.up_191216.txt", sep = ""))
go_mid.down = read.table(paste(in.f, "gowall_ppc_mid.down_191216.txt", sep = ""))

In [93]:
AnnotateTableGO2 <- function(df, celltype){
    
    rownames(df) = NULL
    df$Type_gene_expression_change <- celltype
    df = df[, c("Type_gene_expression_change", "category", "over_represented_pvalue", "under_represented_pvalue", "numDEInCat", "numInCat", "term", "ontology", "pvalue", "bonferroni")]
    return(df)
    
}

In [94]:
t6 = rbind(AnnotateTableGO2(go_up,       "Class_1"),
           AnnotateTableGO2(go_down,     "Class_2"),
           AnnotateTableGO2(go_mid.up,   "Class_3"),
           AnnotateTableGO2(go_mid.down, "Class_4")
     )

In [95]:
write.csv(t6, file = paste(out, "Table_", table_count, ".csv", sep = ""))

# Table 8

Demuxlet results for batch 1 cryopreserved scRNA-seq

In [96]:
table_count = 8

In [97]:
t2 = read.csv("../Table/Table_S2_Demuxlet_Results_V04.csv")

In [98]:
t2 = t2[! t2$Assay == "snATAC_seq", ]

In [99]:
t2$Concordance_rate_singlets <- NULL
t2$Concordance_rate_doublets <- NULL

In [100]:
t2$Assay <- NULL

In [101]:
t2$UDID <- t2$Subject
t2$Subject <- NULL

In [102]:
t2 = t2[, c(3, 1, 2)]

In [103]:
write.csv(t2, file = paste(out, "Table_", table_count, ".csv", sep = ""))

# Table 9

Demuxlet results for batch 2 cryopreserved scRNA-seq

In [104]:
table_count = 9

In [105]:
t3 = read.csv("../Table/Table_S3_Scrublet_Results_V01.csv")

In [106]:
colnames(t3) <- c("Assay", "Assay_UUID", "UDID", "Batch", "Cells","Features", "Threshold", "Detected_doublet_rate",
                 "Estimated_doublet_rate", "Expected_doublet_rate", "Estimated_doublet_rate.1")

In [107]:
t3$Expected_doublet_rate <- NULL
t3$Estimated_doublet_rate.1 <- NULL

In [108]:
t3 = t3[! t3$Assay == "snATAC_seq", ]

In [109]:
t3$Assay <- NULL
t3$Cells <- NULL
t3$Features <- NULL

In [110]:
t3$Estimated_doublet_rate <- NULL

In [111]:
write.csv(t3, file = paste(out, "Table_", table_count, ".csv", sep = ""))